# 프로젝트 목차

#### 1. 데이터 읽기
    - 코로나 데이터 블러오고 DataFrame 구조 확인
    
#### 2. 데이터 정제
    - 비어있거나 쓸모없는 데이터 삭제
    
#### 3. 데이터 시각화

In [23]:
import requests
import pprint
##### open API 활용 신청
# 공공데이터포털에서 보건복지부 코로나19 감염 현황 데이터 활용 신청하기

#인증키 입력
encoding = '발급받은 인코딩 인증키'
decoding = '발급받은 디코딩 인증키'

# url 이해하기
"""
url : API명
serviceKey : 위 상세 페이지에서 발급받은 String 값
pageNo : 페이지 번호
numOfRows : 페이지 내 결과 수
startCreateDt : 검색시작일(YYYYMMDD)
endCreateDt : 검색종료일(YYYYMMDD)
"""

# 2022년 2월 코로나 데이터
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19SidoInfStateJson'
params ={'serviceKey' : decoding, 
         'pageNo' : '1',
         'numOfRows' : '10',
         'startCreateDt' : '20220101',
         'endCreateDt' : '20220217' }

response = requests.get(url, params=params)
          
#xml 내용
response = requests.get(url, params=params)
print(response.content)

#깔끔한 출력을 위한 코드
p=pprint.PrettyPrinter(indent=4)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE ACCESS DENIED ERROR.</resultMsg></header></response>'


In [24]:
### xml을 dataframe으로 변환하기
from os import name
import xml.etree.ElementTree as et
import pandas as pd
import bs4
from lxml import html
from urllib.parse import urlencode, quote_plus, unquote

## 각 컬럼의 값
"""
SEQ : 게시글번호(국내 시도별 발생현황 고유값)
CREATE_DT: 	등록일시분초
DEATH_CNT: 	사망자 수
GUBUN: 	시도명(한글)
GUBUN_CN: 	시도명(중국어)
gubunEn: 시도명(영어)
INC_DEC: 전일대비 증감 수
ISOL_CLEAR_CNT: 격리 해제 수
QUR_RATE: 10만명당 발생률
STD_DAY: 기준일시
UPDATE_DT: 수정일시분초
DEF_CNT: 확진자 수
ISOL_ING_CNT: 격리중 환자수
OVER_FLOW_CNT: 해외유입 수
LOCAL_OCC_CNT: 지역발생 수
"""

#bs4 사용하여 item태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')
print(rows)

[<item><createDt>2022-02-17 08:52:05.139</createDt><deathCnt>7238</deathCnt><decideCnt>1645975</decideCnt><seq>792</seq><stateDt>20220217</stateDt><stateTime>00:00</stateTime><updateDt>2022-02-18 09:05:35.842</updateDt></item>, <item><createDt>2022-02-16 09:20:27.8</createDt><deathCnt>7202</deathCnt><decideCnt>1552843</decideCnt><seq>791</seq><stateDt>20220216</stateDt><stateTime>00:00</stateTime><updateDt>2022-02-17 08:52:33.561</updateDt></item>, <item><createDt>2022-02-15 09:01:44.846</createDt><deathCnt>7163</deathCnt><decideCnt>1462408</decideCnt><seq>790</seq><stateDt>20220215</stateDt><stateTime>00:00</stateTime><updateDt>2022-02-16 09:20:55.138</updateDt></item>, <item><createDt>2022-02-14 09:01:51.395</createDt><deathCnt>7102</deathCnt><decideCnt>1405244</decideCnt><seq>789</seq><stateDt>20220214</stateDt><stateTime>00:00</stateTime><updateDt>2022-02-15 09:02:24.905</updateDt></item>, <item><createDt>2022-02-13 08:50:26.798</createDt><deathCnt>7081</deathCnt><decideCnt>1350627

In [25]:
# 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

# xml 안의 데이터 수집
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

In [ ]:
#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)
print(corona_df.head(19))